# Why Hat
> Analytical approach using Neural Network on tabulate data

The core engine for project $\large\hat{y}$.

The dataset is from [New York City Airbnb Open Data](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv) on kaggle

In [1]:
# default_exp whyhat

In [2]:
# export
import pandas as pd
import numpy as np
from pathlib import Path
import os
import json
from torchember.core import color

In [3]:
DATA = Path("../data")

In [4]:
CSV_PATH = DATA/"AB_NYC_2019.csv"

In [5]:
df = pd.read_csv(CSV_PATH)
df.sample(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
30776,23820538,Large 3 Bedroom Loft in Heart of Williamsburg,28373054,Nadira,Brooklyn,Williamsburg,40.71201,-73.95882,Entire home/apt,600,3,14,2019-06-17,0.98,1,167
19830,15876032,Great Bedroom in a great location. Fun hosts.,59358,V.Stephan,Manhattan,Upper East Side,40.78440,-73.94856,Private room,65,2,42,2019-06-13,1.31,2,0
43112,33451336,Artist Heaven Garden Loft,132099860,Mark,Manhattan,East Village,40.72283,-73.98382,Entire home/apt,450,5,0,NaN,NaN,1,310
15656,12644989,Modern high-end studio!,9997184,Kelly,Manhattan,Upper West Side,40.79376,-73.96734,Entire home/apt,185,2,10,2016-09-01,0.26,1,0
42497,32986796,Furnished private room 5 minutes from Subway,141547523,Mathew,Manhattan,Harlem,40.82943,-73.94690,Private room,50,2,13,2019-06-15,3.42,1,65
28719,22205765,Beautiful spacious Greenwich Village apt w/ ga...,16798487,Vadim,Manhattan,Greenwich Village,40.73506,-73.99743,Entire home/apt,400,7,0,NaN,NaN,1,0
33452,26447297,Northern Manhattan Getaway,319488,Melinda,Manhattan,Inwood,40.86755,-73.92870,Private room,125,2,21,2019-06-23,1.82,1,141
2873,1640995,One bedroom apartment - UES,8313953,Natalie,Manhattan,Upper East Side,40.76866,-73.95360,Entire home/apt,140,6,36,2019-06-06,0.69,1,71
20541,16338454,Cozy & Sunlit Three Bedroom Apartment,5828836,Dana,Manhattan,Harlem,40.82548,-73.94983,Entire home/apt,255,2,1,2017-01-01,0.03,1,0
42490,32980398,"Your own sanctuary! Private Floor, 3min to subway",5338560,Daniella,Brooklyn,Bedford-Stuyvesant,40.67812,-73.91465,Entire home/apt,95,2,14,2019-07-07,4.20,1,275


### Config how we learn the columns

In [6]:
# export
class RichColumn(object):
    """
    A pandas series manager
    """
    def __init__(self,column, is_y = False,min_occur = 5, is_emb = True,hidden_size=50):
        self.col = column
        self.col.rc = self
        self.name = self.col.name
        self.min_occur = min_occur
        self.hidden_size = hidden_size
        self.is_emb =  is_emb
        self.is_y = is_y
        self.use = True
        self.is_conti = True
        self.defined = False
        
    def kill(self):
        """
        set column to kill mode, that it would not be involved in the learning
        """
        self.defined = True
        self.use = False
        
    def conti(self):
        """
        set column to contineous data
        """
        self.defined = True
        self.is_conti = True
        
    def disc(self):
        """
        set column to discrete data
        """
        self.defined = True
        self.is_conti = False
        
    def is_number(self):
        """
        Is this column's data type in any form of number
        """
        return self.col.dtype in (int,float,
                              np.float16,np.float32,np.float64,np.float64,
                              np.int0,np.int8,np.int16,np.int32,np.int64)
    
    def __bool__(self):
        """
        is this column going to join the learning
        """
        return self.use
    
    def __len__(self):
        """
        width of column when entering the model, or used as target
        """
        if self.is_conti:
            return 1
        else:
            if self.is_emb:
                return self.hidden_size
            else:
                width = len(self.top_freq)
                width =1 if width==2 else width
                return width
    
    def __repr__(self,):
        return f"<Rich Column:{self.name}>"
    
    def top_freq_(self):
        freq = self.freq()
        self.top_freq = freq[freq[self.name]>=self.min_occur].reset_index()
        return self.top_freq
    
    def freq(self):
        return pd.DataFrame(data=self.col.value_counts())
    
class RichDF(object):
    """
    A pandas dataframe manager
    """
    def __init__(self,df):
        self.df = df
        self.columns = dict()
        for colname in self.df:
            self.columns.update({colname:RichColumn(df[colname])})
        
    @property
    def col_conf(self):
        return dict((k,{"use":v.use,"is_cont":v.is_conti}) for k,v in self.columns.items())
        
    def kill(self,colname):
        self.df[colname].rc.kill()
        
    def conti(self,colname):
        self.df[colname].rc.conti()
        
    def disc(self,colname):
        self.df[colname].rc.disc()
        
    def set_col(self,rcol):
        if rcol.defined:
            print(f"{rcol.name} defined, use:{rcol.use}, contineus?:{rcol.is_conti}")
        print(color.bold("="*30))
        print(color.cyan(rcol.name))
        print(color.red(f"number? {rcol.is_number()}"))
        print(rcol.top_freq_().head(5))
              
        print(color.red("Is this a [C]ontineous, [D]iscrete or a column we do[N]'t need? default N"))
        x = input().lower()
        if x=="c":
            rcol.conti()
            print(color.blue(f"{rcol.name} set to contineous data"))
        elif x =="d":
            rcol.disc()
            print(color.blue(f"{rcol.name} set to discrite data"))
        elif (x =="") or (x=="n"):
            rcol.kill()
            print(color.blue(f"{rcol.name} will not be involved in learning"))
        else:
            print(color.yellow(f"option [{x}] not found, try Again?"))
        
        
    def tour(self):
        """
        Go through column 1 by 1 to decide the processing for its data
        """
        for colname in self.df:
            col = self.df[colname]
            while col.rc.defined==False:
                self.set_col(col.rc)
                
    def set_y(self, *colnames):
        """
        set columns to y
        all the columns that use==True and is_y==False will be treated as x
        """
        for colname in colnames:
            rc = self.columns[colname]
            rc.is_y = True
            rc.use = True
            
    def set_x(self, *colnames):
        """
        set columns to x
        of course,every columns' default status is x, 
        so you don't have to set this if you accidentally set x to y
        """
        for colname in colnames:
            rc = self.columns[colname]
            rc.use = True

In [7]:
rdf = RichDF(df)

In [8]:
rdf.tour()

id
number? True
Empty DataFrame
Columns: [index, id]
Index: []
Is this a [C]ontineous, [D]iscrete or a column we do[N]'t need? default N

id will not be involved in learning
name
number? False
                          index  name
0                Hillside Hotel    18
1           Home away from home    17
2  New york Multi-unit building    16
3            Brooklyn Apartment    12
4                  Private Room    11
Is this a [C]ontineous, [D]iscrete or a column we do[N]'t need? default N
d
name set to discrite data
host_id
number? True
       index  host_id
0  219517861      327
1  107434423      232
2   30283594      121
3  137358866      103
4   12243051       96
Is this a [C]ontineous, [D]iscrete or a column we do[N]'t need? default N
d
host_id set to discrite data
host_name
number? False
          index  host_name
0       Michael        417
1         David        403
2  Sonder (NYC)        327
3          John        294
4          Alex        279
Is this a [C]ontineous, [D]iscret